In [44]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import os

In [45]:
# .env 파일 로드
from dotenv import load_dotenv
load_dotenv()

True

In [69]:
# LLM 설정
llm = ChatOpenAI(
    model = 'gpt-4o',
    temperature = 0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)
# 대화의 맥락 유지를 위한 memory 설정
memory = ConversationBufferMemory(memory_key="history")

In [70]:
# 프롬프트 1: 회사/직책이 요구하는 것 추출
job_requirements_prompt = PromptTemplate(
    input_variables=['company', 'position'],
    template = (
        '알려주신 회사명 [{company}]와 직무 [{position}]에서,'
        '이 역할에 필요한 핵심 기술, 자질, 책임 등을 식별해 봐.'
    )
)
job_requirements_chain = LLMChain(llm=llm, prompt=job_requirements_prompt, memory=memory)

In [71]:
# 프롬프트 2: 요구되는 것을 바탕으로 면접 질문 생성
interview_questions_prompt = PromptTemplate(
    input_variables = ['requirements'],
    template = (
        '다음의 직무 요구사항에 따라 : {requirements},'
        '이러한 기준을 평가하는 3가지 세부적이고 역할별 면접 질문을 생성합니다.'
    )
)
interview_questions_chain = LLMChain(llm=llm, prompt=interview_questions_prompt, memory=memory)

In [72]:
# 프롬프트 3: 사용자 대답 평가
answer_evaluation_prompt = PromptTemplate(
    input_variables = ['question', 'user_answer'],
    template = (
        '사용자의 질문에 대한 다음 답변을 평가 : {question}\n\n'
        "사용자의 답변 : {user_answer}\n\n"
        '관련성, 심도, 명확성을 기반으로 평가 제공합니다.'
    )
)
answer_evaluation_chain = LLMChain(llm=llm, prompt=answer_evaluation_prompt, memory=memory)


In [73]:
# 프롬프트 4: 모범 답안 생성
model_answer_prompt = PromptTemplate(
    input_variables=['user_answer', 'evaluation'],
    template=(
        '사용자의 주어진 답안 : {user_answer}, 그리고 그에 대한 평가 : {evaluation},'
        '면접 답변에 대한 모범 답안을 반영하여 개선된 답변을 생성합니다.'
    )
)
model_answer_chain = LLMChain(llm=llm, prompt=model_answer_prompt, memory=memory)

In [74]:
# Prompt 통합 flow
def interview_simulation(company, position, user_answers):
    # 1. 회사/직책 요구사항 추출
    requirements = job_requirements_chain.apply([{"company": company, "position": position}])[0]["text"]
    print(f"Job Requirements:\n{requirements}\n")

    # 2. 면접 질문 생성
    questions = interview_questions_chain.apply([{"requirements": requirements}])[0]["text"]
    print(f"Generated Questions:\n{questions}\n")

    # 3. 사용자 답변 평가 및 모범 답안 생성
    for idx, (question, answer) in enumerate(zip(questions.split('\n'), user_answers), 1):
        evaluation = answer_evaluation_chain.apply([{"question": question, "user_answer": answer}])[0]["text"]
        print(f"Evaluation for Answer {idx}:\n{evaluation}\n")
        
        model_answer = model_answer_chain.apply([{"user_answer": answer, "evaluation": evaluation}])[0]["text"]
        print(f"Model Answer for Question {idx}:\n{model_answer}\n")

In [ ]:
# 사용 예시
company_name = "OpenAI"
position_name = "개발 엔지니어"
user_responses = [
    "저는 머신 러닝 모델을 구축하고 배포한 경험이 있습니다.",
    "저는 팀워크와 소통이 제 가장 강력한 자산이라고 믿습니다.",
    "저는 새로운 기술을 배우고 적응하는 데 열정적입니다."
]
interview_simulation(company_name, position_name, user_responses)

Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Parent run e073b183-dc8d-435c-b669-6d6e95ffcd32 not found for run 03589010-f298-4f43-a2f7-b72509839b54. Treating as a root run.
Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID e073b183-dc8d-435c-b669-6d6e95ffcd32.')
Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Parent run 951376c7-4e42-41d4-a7db-2d1e3ad30fc6 not found for run 19f19a7d-7743-4df7-834e-8b8d7addb611. Treating as a root run.


Job Requirements:
OpenAI에서 개발 엔지니어로 일하기 위해 필요한 핵심 기술, 자질, 그리고 책임은 다음과 같습니다.

### 핵심 기술
1. **프로그래밍 언어**: Python, C++, JavaScript 등과 같은 주요 프로그래밍 언어에 대한 깊은 이해와 숙련도.
2. **머신러닝 및 인공지능**: 머신러닝 라이브러리(TensorFlow, PyTorch 등)와 AI 모델 개발에 대한 경험.
3. **데이터 처리 및 분석**: 대량의 데이터를 처리하고 분석할 수 있는 능력, SQL 및 NoSQL 데이터베이스 경험.
4. **소프트웨어 개발**: 소프트웨어 개발 라이프사이클에 대한 이해, 버전 관리 시스템(Git 등) 사용 경험.
5. **클라우드 컴퓨팅**: AWS, Google Cloud, Azure 등의 클라우드 플랫폼 경험.

### 자질
1. **문제 해결 능력**: 복잡한 문제를 분석하고 해결할 수 있는 능력.
2. **창의성**: 새로운 아이디어를 제시하고 혁신적인 솔루션을 개발할 수 있는 능력.
3. **협업 능력**: 다양한 팀과 효과적으로 협업할 수 있는 커뮤니케이션 스킬.
4. **자기 주도성**: 프로젝트를 독립적으로 추진하고 관리할 수 있는 능력.
5. **학습 능력**: 빠르게 변화하는 기술 트렌드에 적응하고 지속적으로 학습할 수 있는 능력.

### 책임
1. **소프트웨어 및 시스템 설계**: 고품질의 소프트웨어와 시스템을 설계하고 개발.
2. **AI 모델 개발 및 개선**: AI 및 머신러닝 모델의 연구, 개발, 테스트, 배포를 수행.
3. **코드 품질 유지**: 코드 리뷰를 통해 코드 품질을 유지하고 개선.
4. **프로젝트 관리**: 프로젝트의 목표와 일정을 관리하고, 진척 상황을 보고.
5. **기술 문서 작성**: 개발 과정 및 결과를 문서화하여 공유.

이 역할은 지속적인 기술 발전과 문제 해결을 요구하며, OpenAI의 목표와 비전에 맞춰 혁신적인 AI 솔루션을 개발하는 데 중점을 둡니다.



Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID 951376c7-4e42-41d4-a7db-2d1e3ad30fc6.')
Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Parent run 19e2309f-8b72-4b52-b03b-b8b10d228d74 not found for run 829adc7d-ba50-458c-8096-2874ce130b0d. Treating as a root run.


Generated Questions:
OpenAI에서 개발 엔지니어로서의 역량을 평가하기 위한 역할별 면접 질문은 다음과 같습니다.

### 프로그래밍 언어 및 소프트웨어 개발
1. **질문**: 이전 프로젝트에서 Python과 C++를 사용하여 복잡한 소프트웨어 시스템을 개발한 경험에 대해 설명해 주세요. 해당 프로젝트의 주요 도전 과제는 무엇이었으며, 이를 어떻게 해결하셨나요?
   - **의도**: 지원자의 프로그래밍 언어에 대한 깊은 이해와 소프트웨어 개발 경험을 평가하기 위함입니다.

### 머신러닝 및 AI 모델 개발
2. **질문**: TensorFlow 또는 PyTorch를 사용하여 AI 모델을 개발한 경험이 있습니까? 해당 프로젝트에서 선택한 알고리즘과 모델의 성능을 개선하기 위한 접근 방식에 대해 설명해 주세요.
   - **의도**: 지원자의 머신러닝 라이브러리 활용 능력과 AI 모델 개발 및 개선 능력을 평가하기 위함입니다.

### 협업 및 프로젝트 관리
3. **질문**: 과거에 다양한 팀과 협력하여 수행한 프로젝트 중 가장 성공적이었던 경험을 공유해 주세요. 귀하의 역할은 무엇이었고, 프로젝트의 성공에 기여한 주요 요인은 무엇이라고 생각하십니까?
   - **의도**: 지원자의 협업 능력, 의사소통 스킬, 그리고 프로젝트 관리 역량을 평가하기 위함입니다.

이러한 질문들은 지원자가 OpenAI에서 개발 엔지니어로서의 역할을 수행하는 데 필요한 기술과 자질을 갖추고 있는지를 평가하는 데 중점을 둡니다.



Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID 19e2309f-8b72-4b52-b03b-b8b10d228d74.')
Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Parent run dc864be6-ba3c-4701-a618-e41a028c4c3a not found for run d9a152aa-61da-4cb9-b87e-0b3638ab67a2. Treating as a root run.


...

Model Answer for Question 2:
저는 팀워크와 소통이 제 가장 강력한 자산이라고 믿습니다. 예를 들어, 이전 직장에서 진행했던 프로젝트에서 팀원들과의 원활한 소통을 통해 프로젝트 목표를 명확히 하고, 각자의 역할을 효과적으로 분배할 수 있었습니다. 이 과정에서 발생한 문제들을 적극적으로 해결함으로써 프로젝트를 성공적으로 완수할 수 있었습니다. 이러한 경험을 통해 팀워크와 소통이 어떻게 성과로 이어지는지를 깊이 이해하게 되었으며, 이러한 역량을 통해 팀의 목표 달성에 기여할 수 있다고 확신합니다.



Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID a0bbed10-d20c-447a-9c4c-2cdf283c83e6.')
Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Parent run 31e1dfe3-4134-4518-9615-a7fd6b8a3ca0 not found for run a3ae9c33-8908-4683-ab53-ea605a8a61cd. Treating as a root run.


Evaluation for Answer 3:
사용자의 답변을 평가하면 다음과 같습니다:

1. **관련성**: 사용자의 답변은 프로그래밍 언어 및 소프트웨어 개발과 관련된 질문에 대해 어느 정도 관련성이 있습니다. 새로운 기술을 배우고 적응하는 것은 소프트웨어 개발자에게 중요한 자질이며, 이는 프로그래밍 언어 및 소프트웨어 개발 분야와 연관이 있습니다. 하지만 질문이 더 구체적인 경우, 예를 들어 특정 프로그래밍 언어나 개발 방법론에 대한 것이었다면, 답변의 관련성이 떨어질 수 있습니다.

2. **심도**: 답변은 다소 일반적이며 심도 있는 정보를 제공하지 않습니다. "새로운 기술을 배우고 적응하는 데 열정적입니다"라는 문장은 개인의 학습 태도를 나타내지만, 구체적인 기술이나 경험, 또는 학습 방법에 대한 정보가 부족합니다. 보다 심도 있는 답변을 위해서는 사용자가 어떤 기술을 배우고 있는지, 어떻게 배우고 있는지, 또는 어떤 경험이 있는지를 추가하면 좋겠습니다.

3. **명확성**: 답변은 명확하게 작성되어 있으며, 사용자의 학습 태도를 잘 전달하고 있습니다. 간결하고 이해하기 쉬운 문장으로 되어 있어 명확성 면에서는 문제가 없습니다.

종합적으로, 사용자의 답변은 기본적인 관련성과 명확성을 갖추고 있지만, 심도 면에서 개선이 필요합니다. 보다 구체적인 정보나 예시를 추가하면 더 나은 답변이 될 수 있습니다.



Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID 31e1dfe3-4134-4518-9615-a7fd6b8a3ca0.')


Model Answer for Question 3:
개선된 답변:

"저는 새로운 기술을 배우고 적응하는 데 열정적입니다. 예를 들어, 최근에는 Python 프로그래밍 언어를 독학으로 공부하며 데이터 분석 프로젝트를 진행했습니다. 이를 통해 데이터 처리 및 시각화에 대한 이해도를 높일 수 있었습니다. 또한, Git과 같은 버전 관리 시스템을 사용하여 협업 프로젝트에 참여한 경험이 있습니다. 이러한 경험을 통해 새로운 기술을 빠르게 습득하고 실무에 적용하는 능력을 향상시켰습니다. 앞으로도 변화하는 기술 트렌드를 지속적으로 학습하며, 팀과 조직에 기여할 수 있는 역량을 키워 나가고 싶습니다."

이 답변은 사용자의 학습 태도와 함께 구체적인 기술과 경험을 강조하여 심도 있는 정보를 제공합니다.

